In [2]:
import json
import pandas as pd

In [3]:
def export_file(filename):
    print(f'Exporting {filename}')
    with open(f'SERIAL_CAPTURE/{filename}.json') as f:
        d = json.load(f)


    def strip_data(packet, len):
        hex = packet.split(":")
        hex = hex[-len:]
        return ''.join(hex)

    data = []

    for packet in d:

        data_len = int(packet['_source']['layers']['usb']['usb.data_len'])

        packet_str = packet['_source']['layers']['ftdi-ft']

        time = packet['_source']['layers']['frame']['frame.time_relative']

        elem = {}

        if "ftdi-ft.if_a_tx_payload" in packet_str:
            #tx
            elem = {
            "time": time,
            "dir": "tx",
            "packet": strip_data(packet_str["ftdi-ft.if_a_tx_payload"], data_len),
            "len": data_len,
            }
        else:
            #rx
            elem = {
            "time": time,
            "dir": "rx",
            "packet": strip_data(packet_str["ftdi-ft.if_a_rx_payload"], data_len),
            "len": data_len,
            }

        data.append(elem)

    df = pd.DataFrame(data)

    # Create an empy dataframe with the same columns as df
    df_concat = pd.DataFrame(columns=df.columns)

    #itterate over df 

    # COMINE PACKETS
    packet = ""
    length = len(df)
    for index, row in df.iterrows():
        direction = row['dir']
        packet = packet + row['packet']


        if index == length - 1:
            # Over 2 as two chars per byte
            df_row = pd.DataFrame([[row['time'], row['dir'], packet, int(len(packet)/2)]], columns=df.columns)
            df_concat = pd.concat([df_concat, df_row])
            packet = ""
            break
            

        if df.iloc[index+1]['dir'] != direction:
            df_row = pd.DataFrame([[row['time'], row['dir'], packet, int(len(packet)/2)]], columns=df.columns)
            df_concat = pd.concat([df_concat, df_row])
            packet = ""
        

    df_concat.to_csv(f'SERIAL_CAPTURE/{filename}.csv', index=False)

In [4]:
#get list of file in directory with extension json and remove extension

# WIRESHARK FILTER -> ftdi-ft&&(ftdi-ft.if_a_rx_payload||ftdi-ft.if_a_tx_payload)
import os
files = [f.split('.')[0] for f in os.listdir('SERIAL_CAPTURE') if f.endswith('.json')]

for file in files:
    export_file(file)

Exporting capp_press_prog_nobean
Exporting button_press_mocha
Exporting button_press_other
Exporting button_press_white_coffee
Exporting button_press_cappuccino
Exporting button_press_hot_chocolate
Exporting americano_press_prog_goodrun_then_nobeans
Exporting capp_press_prog_goodrun
Exporting americano_press
Exporting button_press_latte_works
Exporting button_press_hot_water
Exporting button_press_latte


In [5]:
tx = "f01700050001000000010002490100000000000000000000000154"


n= 2
chunks = [tx[i:i+n] for i in range(0, len(tx), n)]

#print(chunks)
dec = []
for byte in chunks:
    dec.append(int(byte, 16))

print(dec)


[240, 23, 0, 5, 0, 1, 0, 0, 0, 1, 0, 2, 73, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 84]


In [20]:
import struct

# byte to value conversions

def convert_b_signed(byte):
    return struct.unpack('b', bytes.fromhex(byte))[0]

def convert_b_unsigned(byte):
    return struct.unpack('B', bytes.fromhex(byte))[0]

def convert_b_char(byte):
    return struct.unpack('c', bytes.fromhex(byte))[0]


def convert_unsigned(hex):
    dec = int(hex, 16)
    return dec

def convert_signed(hex):
    bits = len(hex) * 4
    dec = int(hex, 16)
    if dec > (bits-1)**2:
        dec = dec - bits**2
    return dec

def convert_2b_float(byte_2):
    return struct.unpack('e', bytes.fromhex(byte_2))[0]

def convert_4b_float(byte_4):
    return struct.unpack('f', bytes.fromhex(byte_4))[0]

def convert_8b_double(byte_8):
    return struct.unpack('d', bytes.fromhex(byte_8))[0]

def convert_string(byte):
    return struct.unpack('s', bytes.fromhex(byte))[0]


def convert_byte(byte):
    char = convert_b_char(byte)
    ub = convert_unsigned(byte)
    sb = convert_signed(byte)

    return [sb , ub, char]

def convert_2b(byte_2):
    u2 = convert_unsigned(byte_2)
    s2 = convert_signed(byte_2)
    float_s = convert_2b_float(byte_2)
    return [s2, u2, float_s ]

def convert_4b(byte_4):
    u4 = convert_unsigned(byte_4)
    s4 = convert_signed(byte_4)
    f = convert_4b_float(byte_4)

    return [ s4, u4, f]

def convert_8b(byte_8):
    u8 = convert_unsigned(byte_8)
    s8 = convert_signed(byte_8)
    double = convert_8b_double(byte_8)

    return [s8, u8, double]


def window_hexstring(data):
    b =  [data[i:i+2] for i in range(0, len(data) - 1)]
    b2 = [data[i:i+4] for i in range(0, len(data) - 3)]
    b4 = [data[i:i+8] for i in range(0, len(data) - 7)]
    b8 =  [data[i:i+16] for i in range(0, len(data) - 15)]

    b_conversion = pd.DataFrame(columns=['byte', 'signed', 'unsigned', 'char'])
    b2_conversion = pd.DataFrame(columns=['2byte', 'signed_h', 'unsigned_h',  'float_s'])
    b4_conversion = pd.DataFrame(columns=['4byte', 'signed', 'unsigned' , 'float'])
    b8_conversion = pd.DataFrame(columns=['8byte', 'signed_l', 'unsigned_l', 'double'])

    string = ""# convert_string(data)

    for i, byte in enumerate(b):
        b_conversion.loc[i] = [byte] + convert_byte(byte)

    for i, byte_2 in enumerate(b2):
        b2_conversion.loc[i] = [byte_2] + convert_2b(byte_2)
    
    for i, byte_4 in enumerate(b4):
        b4_conversion.loc[i] = [byte_4] + convert_4b(byte_4)
    
    for i, byte_8 in enumerate(b8):
        b8_conversion.loc[i] = [byte_8] + convert_8b(byte_8)

    return b_conversion, b2_conversion, b4_conversion, b8_conversion, string


cap = "f01f0003000100000001005053000001000000000000000000013801790054000502b7"
ame = "f01a000300010000000100108b010000000000000000000000de1f01009f"

b, b2, b4, b8, s = window_hexstring(cap)

print(b)
print(b2)
print(b4)
print(b8)
print(s)




   byte  signed  unsigned     char
0    f0     176       240  b'\xf0'
1    01       1         1  b'\x01'
2    1f      31        31  b'\x1f'
3    f0     176       240  b'\xf0'
4    00       0         0  b'\x00'
..  ...     ...       ...      ...
64   05       5         5  b'\x05'
65   50      16        80     b'P'
66   02       2         2  b'\x02'
67   2b      43        43     b'+'
68   b7     119       183  b'\xb7'

[69 rows x 4 columns]
   2byte  signed_h  unsigned_h      float_s
0   f01f     61215       61471     0.007751
1   01f0       240         496 -8200.000000
2   1f00      7680        7936     0.000002
3   f000     61184       61440     0.000014
4   0003         3           3     0.000046
..   ...       ...         ...          ...
62  0005         5           5     0.000076
63  0050        80          80    32.000000
64  0502      1026        1282     0.000031
65  502b     20267       20523     0.057129
66  02b7       439         695    -0.437988

[67 rows x 4 columns]
      